In [15]:
from prophet import Prophet
from yahooquery import Ticker
import pandas as pd
from datetime import datetime
import pandas_ta as ta
import plotly.express as px
import plotly.graph_objs as go
from prophet.diagnostics import cross_validation
from prophet.diagnostics import performance_metrics

In [24]:
# Nome da ação
symbol = "itsa4.sa"
stock = Ticker(symbol)

history = stock.history(period="120mo")

history.reset_index(level=["symbol"], inplace=True)

history['date'] = history.index

history.set_index(pd.DatetimeIndex(history.index), inplace=True)

data = history[['date','adjclose']].copy();

data.ta.ema(close='adjclose', length=21, append=True)

data.dropna(inplace=True)

In [25]:
fig = px.line(data, x='date', y='adjclose')

fig.update_xaxes(
    rangeslider_visible=True,
    rangeselector=dict(
        buttons=list([
            dict(count=1, label="1m", step="month", stepmode="backward"),
            dict(count=6, label="6m", step="month", stepmode="backward"),
            dict(count=1, label="YTD", step="year", stepmode="todate"),
            dict(count=1, label="1y", step="year", stepmode="backward"),
            dict(step="all")
        ])
    )
)

In [26]:
df_train = data[['date','adjclose','EMA_21']]

df_train = df_train.rename(columns={"date": "ds", "adjclose": "y"})

m = Prophet(daily_seasonality=True)
 
m.fit(df_train)

future = m.make_future_dataframe(periods=50)

future['day'] =  future['ds'].dt.weekday
future = future[future['day'] <=4]

forecast = m.predict(future)

15:27:30 - cmdstanpy - INFO - Chain [1] start processing
15:27:31 - cmdstanpy - INFO - Chain [1] done processing


In [27]:
fig = go.Figure([
    go.Scatter(x=df_train['ds'], y=df_train['y'], name='Atualidade', mode='lines'),
    go.Scatter(x=forecast['ds'], y=forecast['yhat'], name='Previsao', mode='lines'),
    
])

fig.update_xaxes(
    rangeslider_visible=True,
    rangeselector=dict(
        buttons=list([
            dict(count=1, label="1d", step="day", stepmode="backward"),
            dict(count=1, label="1m", step="month", stepmode="backward"),
            dict(count=6, label="6m", step="month", stepmode="backward"),
            dict(count=1, label="YTD", step="year", stepmode="todate"),
            dict(count=1, label="1y", step="year", stepmode="backward"),
            dict(step="all")
        ])
    )
)
fig.show()

In [28]:
pred_df = forecast[forecast['ds'] > datetime.today()][['ds','yhat']]

pred_df.reset_index(inplace=True)

pred_df.drop(labels='index', axis=1,inplace=True)

pred_df.rename(columns={'ds': 'date', 'yhat': 'predicted price'}, inplace=True)

pred_df

,date,predicted price
0,2022-07-04,9.718462
1,2022-07-05,9.724261
2,2022-07-06,9.747148
3,2022-07-07,9.763737
4,2022-07-08,9.788805
5,2022-07-11,9.854159
6,2022-07-12,9.863373
7,2022-07-13,9.888329
8,2022-07-14,9.905621
9,2022-07-15,9.930027


In [29]:
df_cv = cross_validation(m, initial='720 days', period='30 days', horizon = '365 days')

print(df_cv.head())

df_p = performance_metrics(df_cv)
print(df_p.head())

  0%|          | 0/85 [00:00<?, ?it/s]

15:27:45 - cmdstanpy - INFO - Chain [1] start processing
15:27:46 - cmdstanpy - INFO - Chain [1] done processing
15:27:47 - cmdstanpy - INFO - Chain [1] start processing
15:27:47 - cmdstanpy - INFO - Chain [1] done processing
15:27:48 - cmdstanpy - INFO - Chain [1] start processing
15:27:49 - cmdstanpy - INFO - Chain [1] done processing
15:27:50 - cmdstanpy - INFO - Chain [1] start processing
15:27:50 - cmdstanpy - INFO - Chain [1] done processing
15:27:52 - cmdstanpy - INFO - Chain [1] start processing
15:27:52 - cmdstanpy - INFO - Chain [1] done processing
15:27:53 - cmdstanpy - INFO - Chain [1] start processing
15:27:53 - cmdstanpy - INFO - Chain [1] done processing
15:27:55 - cmdstanpy - INFO - Chain [1] start processing
15:27:55 - cmdstanpy - INFO - Chain [1] done processing
15:27:57 - cmdstanpy - INFO - Chain [1] start processing
15:27:57 - cmdstanpy - INFO - Chain [1] done processing
15:27:58 - cmdstanpy - INFO - Chain [1] start processing
15:27:58 - cmdstanpy - INFO - Chain [1]

15:29:54 - cmdstanpy - INFO - Chain [1] done processing
15:29:55 - cmdstanpy - INFO - Chain [1] start processing
15:29:56 - cmdstanpy - INFO - Chain [1] done processing
15:29:57 - cmdstanpy - INFO - Chain [1] start processing
15:29:57 - cmdstanpy - INFO - Chain [1] done processing
15:29:59 - cmdstanpy - INFO - Chain [1] start processing
15:29:59 - cmdstanpy - INFO - Chain [1] done processing
15:30:01 - cmdstanpy - INFO - Chain [1] start processing
15:30:01 - cmdstanpy - INFO - Chain [1] done processing
15:30:03 - cmdstanpy - INFO - Chain [1] start processing
15:30:03 - cmdstanpy - INFO - Chain [1] done processing
15:30:05 - cmdstanpy - INFO - Chain [1] start processing
15:30:05 - cmdstanpy - INFO - Chain [1] done processing
15:30:07 - cmdstanpy - INFO - Chain [1] start processing
15:30:07 - cmdstanpy - INFO - Chain [1] done processing
15:30:09 - cmdstanpy - INFO - Chain [1] start processing
15:30:09 - cmdstanpy - INFO - Chain [1] done processing
15:30:10 - cmdstanpy - INFO - Chain [1] 

          ds      yhat  yhat_lower  yhat_upper         y     cutoff
0 2014-08-08  4.428489    4.308063    4.546365  4.267409 2014-08-07
1 2014-08-11  4.459430    4.337298    4.582486  4.348780 2014-08-07
2 2014-08-12  4.461358    4.343788    4.582839  4.348780 2014-08-07
3 2014-08-13  4.464067    4.343435    4.583877  4.249328 2014-08-07
4 2014-08-14  4.476442    4.355178    4.588359  4.327764 2014-08-07
  horizon       mse      rmse       mae      mape     mdape     smape  \
0 37 days  1.164197  1.078980  0.735515  0.098105  0.074237  0.094696   
1 38 days  1.191314  1.091473  0.743736  0.099392  0.074900  0.095871   
2 39 days  1.217072  1.103210  0.753061  0.100775  0.075976  0.097179   
3 40 days  1.240471  1.113765  0.761602  0.101992  0.076824  0.098321   
4 41 days  1.260511  1.122725  0.769746  0.103134  0.078086  0.099404   

   coverage  
0  0.343291  
1  0.338820  
2  0.332467  
3  0.327138  
4  0.321375  
